# Laboratorio 2

## Análisis y Manipulación de Datos Geoespaciales con GeoPandas

Ayudante: Janus Leonhardt | jaleonhardt@uc.cl

Profesor: Ricardo Hurtubia | rhurtubia@uc.cl

## Objetivos del Laboratorio:

1. Profundizar en la manipulación de datos geoespaciales con GeoPandas a partir de lo revisado en el Laboratorio 1, incluyendo las operaciones básicas de unión espacial, filtros y agregaciones.

2. Aprender a unir información del Censo 2017 y del Sistema de Indicadores y Estándares de Desarrollo Urbano (SIEDU) con los datos catastrales del SII.

3. Consolidar un archivo geográfico de celdas con información agregada que integra variables catastrales y datos externos del Censo y SIEDU.

4. Revisar algunos métodos de visualización de datos geoespaciales para identificar patrones relevantes a nivel de celdas.

## 1. Repaso del Laboratorio 1

En el **Laboratorio 1**, aprendimos los conceptos básicos de GIS e introducimos la base de datos del "Detalle Catastral de Bienes Raíces" del Servicio de Impuestos Internos (SII) de Chile, filtrado para Puerto Montt. 

### Resumen de los Pasos en el Lab 1:

1. **Carga y Exploración de Datos**: Usamos `GeoPandas` para leer y mostrar los archivos GeoJSON del catastro del SII. 

2. **Reindexación H3**: Se aplicó la librería `h3` para regenerar un índice hexagonal geoespacial a una resolución 9.

3. **Agrupación por Celda**: Se mostraron métodos para agrupar información catastral por *h3*, en particular respecto a los metros cuadrados y unidades por tipos de uso de suelo.

4. **Exportación de Datos**: Se guardaron los GeoDataFrames resultantes (`lab01/agentes.geojson` y `lab01/celdas.geojson`) para su uso en este laboratorio.

---

## 2. Instalación y Configuración del Entorno de Trabajo

### Requisitos Previos

- Python 3.7 o superior

- Jupyter Notebook

- **Librerías de Python**:

  - pandas  
  - geopandas  
  - shapely  
  - h3  
  - matplotlib  
  - folium  
  - mapclassify  
  - **seaborn**

### Instalación de Librerías

Recordemos que las librerías deben haber quedado correctamente instaladas en nuestro entorno virtual (`.conda` o `.venv`) creado en el Laboratorio 1. En caso contrario, se debe ejecutar la siguiente línea de código.

`!pip install pandas geopandas shapely h3 matplotlib folium mapclassify seaborn`

<br>

In [ ]:
# instalar las librerías faltantes
!pip install seaborn

---

## 3. Carga de Datos Previos

En este laboratorio, partiremos desde los archivos generados en el Laboratorio 1, los cuales contienen la información catastral relacionada con celdas *h3*.

- **`agentes.geojson`**: Contiene información detallada de cada línea de construcción (agente) en Puerto Montt, con un índice *h3*.

- **`celdas.geojson`**: Contiene información agregada por cada celda *h3*, incluyendo la geometría hexagonal y atributos del entorno construido que se obtienen de la agregación de las líneas de construcción.

<br>

In [ ]:
import pandas as pd
import geopandas as gpd

# (opcional) ajustes de visualización
pd.set_option('display.max_columns', None)

# cargar el catastro a nivel de líneas de construcción indexado por H3
agentes = gpd.read_file("../lab_01/agentes.geojson")

# cargar la capa de celdas con información agregada
celdas = gpd.read_file("../lab_01/celdas.geojson")

In [ ]:
# inspeccionar las primeras filas de la tabla de agentes
agentes.head()

In [ ]:
# inspeccionar las primeras filas de la tabla de celdas
celdas.head()

In [ ]:
# revisamos el sistema de referencia de coordenadas de ambas capas
print("CRS de catastro_h3:", agentes.crs)
print("CRS de celdas_h3:", celdas.crs)

---

## 4. Introducción a los Datos del Censo 2017

En Chile, el **Censo de Población y Vivienda 2017** es una fuente esencial de datos sociodemográficos y de uso de viviendas/hogares. Durante la planificación y ejecución, el **INE (Instituto Nacional de Estadísticas)** utiliza una zonificación censal diferente de la división del SII, basada en lo siguiente.

1. **Manzanas Urbanas**: Agrupan o dividen el territorio urbano en manzanas, incluyendo variables como población total, número de viviendas, etc.

2.	**Entidades Rurales**: Segmentan el área rural (pueblos, aldeas, sectores con menor densidad de construcción), incluyendo variables de caracterización similares a las manzanas urbanas.

<br>

<img src="assets/censo.png" alt="SII" style="height: 150px; width: auto; display: block; margin-top: 5px;">

<br>

In [ ]:
# cargar manzanas urbanas y entidades rurales
manzanas_censales = gpd.read_file("manzanas_censales.geojson")
entidades_censales = gpd.read_file("entidades_censales.geojson")

# filtrar las manzanas y entidades de Puerto Montt
manzanas_censales = manzanas_censales[manzanas_censales['COMUNA'] == 'PUERTO MONTT']
entidades_censales = entidades_censales[entidades_censales['NOMBRE_COM'] == 'PUERTO MONTT'] 

In [ ]:
# inspeccionar las primeras filas de la tabla de manzanas
manzanas_censales.head()

In [ ]:
# inspeccionar las primeras filas de la tabla de entidades
entidades_censales.head()

In [ ]:
# verificar y unificar CRS con celdas_h3
print("CRS Manzanas Censales:", manzanas_censales.crs)
print("CRS Entidades Censales:", entidades_censales.crs)

In [ ]:
# filtrar y unificar el nombre de las columnas de manzanas y entidades
manzanas_censales = manzanas_censales[['PERSONAS', 'TOTAL_VIVI', 'TOT_HOGARE', 'geometry']]
entidades_censales = entidades_censales[['TOTAL_PERSONAS', 'TOTAL_VIVIENDAS', 'CANTIDAD_HOGARES', 'geometry']]

# modificamos los nombres de las columnas
manzanas_censales.columns = ['personas', 'viviendas', 'hogares', 'geometry']
entidades_censales.columns = ['personas', 'viviendas', 'hogares', 'geometry']

# transformamos "personas" a valores enteros
entidades_censales['personas'] = entidades_censales['personas'].astype(int)

# unir manzanas y entidades
censo = gpd.GeoDataFrame(pd.concat([manzanas_censales, entidades_censales], ignore_index=True))

In [ ]:
# mostrar el GeoDataFrame resultante
censo.explore(column='viviendas', scheme='NaturalBreaks', k=10)

In [ ]:
# modificamos el CRS del GeoDataFrame a UTM 19S (EPSG:32719)
censo = censo.to_crs(epsg=32719)

# calculamos la densidad poblacional
censo['densidad_poblacional'] = censo['personas'] / censo['geometry'].area

# volvemos al CRS WGS84 (EPSG:4326)
censo = censo.to_crs(epsg=4326) 

In [ ]:
# mostrar el GeoDataFrame resultante
censo.explore(column='densidad_poblacional', scheme='NaturalBreaks', k=10)

---

## 5. Ejemplo de Integración de Datos Censales

Ahora, el objetivo es **transferir la información de personas, viviendas y hogares** (proveniente de manzanas y/o entidades censales) a nuestras celdas. Para ello, **usaremos `overlay`** de **GeoPandas**, que nos permitirá calcular la **proporción de área** superpuesta entre cada polígono censal y las celdas, asignando así de forma proporcional las variables de interés (personas, viviendas, hogares).

### Lógica del `overlay` en GeoPandas

En GeoPandas, el método `overlay` se enfoca en crear nuevas geometrías basadas en la intersección (u otros mecanismos) de dos GeoDataFrames. En particular, las operaciones soportadas por `overlay` en GeoPandas son las siguientes.

- **`intersection`**: Identifica las geometrías comunes entre ambos GeoDataFrames.

- **`union`**: Combina todas las geometrías y atributos.

- **`difference`**: Determina las geometrías de un GeoDataFrame que no se superponen con el otro.

- **`symmetric_difference`**: Retorna las geometrías no comunes a ambos GeoDataFrames.

**Ejemplo**: Encontrar la intersección de dos conjuntos de polígonos.

`result = gpd.overlay(polygons1_gdf, polygons2_gdf, how="intersection")`

### Integración de Área por Proporción de Área

Para manejar casos donde una celda se sobrepone a múltiples polígonos, en este caso las manzanas y entidades censales, aplicaremos una proporción de área para ponderar la contribución de cada polígono censal a la celda.

Sea $P_{c}$ el conjunto de polígonos censales que se intersectan con la celda c, que podemos determinar a través del método `overlay` de GeoPandas; sea $A_{cp}$ el área de la intersección entre la celda c y el polígono p; sea $A_{p}$ el área total del polígono censal p; y $q_{p}^{v}$  el valor de la variable v en el polígono p (p. ej., personas, viviendas, hogares). Luego, el valor de la variable v en la celda c será:

$q_{c}^{v} = \sum_{p \in P_{c}} \left( \frac{A_{cp}}{A_{p}} \right) \cdot q_{p}^{v}$

Esta fórmula asegura que la integración de los datos censales a las celdas se realice de manera proporcional al área de superposición, evitando así sesgos derivados de diferencias en tamaños y formas de los polígonos censales y las celdas.

<br>


In [ ]:
# reproyectar a un CRS proyectado para cálculos de área precisos
celdas_temp = celdas.to_crs(epsg=32719)
censo_temp = censo.to_crs(epsg=32719)

# agregar campo de área a los polígonos censales
censo_temp['area_poligono'] = censo_temp.geometry.area

# realizar overlay con how='intersection'
celdas_censo_overlay = gpd.overlay(celdas_temp, censo_temp, how='intersection')

# calcular el área proporcional de cada intersección (A_cp / A_p)
celdas_censo_overlay['area_proporcional'] = celdas_censo_overlay.geometry.area / celdas_censo_overlay['area_poligono']

# ajustar variables censales multiplicándolas por 'area_proporcional'
celdas_censo_overlay['personas'] = celdas_censo_overlay['personas'] * celdas_censo_overlay['area_proporcional']
celdas_censo_overlay['viviendas'] = celdas_censo_overlay['viviendas'] * celdas_censo_overlay['area_proporcional']
celdas_censo_overlay['hogares'] = celdas_censo_overlay['hogares'] * celdas_censo_overlay['area_proporcional']

# agrupar por h3 para sumar las contribuciones de cada intersección
celdas_censo_overlay = celdas_censo_overlay.groupby('h3', as_index=False).agg({
    'personas':'sum', 'viviendas':'sum', 'hogares':'sum'})

# unir con el GeoDataFrame base de celdas
celdas_censo = celdas.merge(celdas_censo_overlay, on='h3', how='left')

# reproyectar a EPSG:4326 para visualización
celdas_censo = celdas_censo.to_crs(epsg=4326)

In [ ]:
# visualizar la variable 'hogares' en las celdas
celdas_censo.explore(column='hogares', scheme='NaturalBreaks', k=10)

---

## 6. Introducción al Sistema de Indicadores y Estándares de Desarrollo Urbano (SIEDU)

El **SIEDU** es una iniciativa conjunta entre el **Consejo Nacional de Desarrollo Urbano (CNDU)** y el **Instituto Nacional de Estadísticas (INE)** para **medir la calidad de vida** en las ciudades chilenas.

Los indicadores del SIEDU se obtienen en gran medida a partir del esfuerzo del INE por georreferenciar los llamados **Bienes Públicos Urbanos** (comisarías de carabineros, paraderos de transporte público, centros de salud primaria, establecimientos de educación básica e inicial, plazas y parques). Estos se representan como **puntos georreferenciados**, y en este laboratorio, en particular, nos enfocaremos en los **paraderos de transporte público** como un ejemplo de integración de datos en nuestras celdas con la zonificación *h3*.

<br>

<img src="assets/siedu.png" alt="SII" style="height: 150px; width: auto; display: block; margin-top: 5px;">

<br>

In [ ]:
# cargar la capa de paraderos y filtrar por comuna
paraderos = gpd.read_file("paraderos.geojson")
paraderos = paraderos[paraderos['comuna']==10101]

# visualizar la capa de paraderos
paraderos.explore()

---

## 7. Ejemplo de Integración de Paraderos

Como parte de la información derivada del SIEDU (o de otras fuentes), queremos **contar cuántos paraderos** hay en cada celda H3. Para esto, utilizaremos la **unión espacial** mediante `sjoin`, dado que tenemos **puntos** (paraderos) y **polígonos** (celdas).

### 7.1. Lógica de `sjoin` en GeoPandas

En GeoPandas, el método `sjoin`  se enfoca en unir dos conjuntos de datos geoespaciales basándose en una relación espacial entre sus geometrías. En particular, las operaciones soportadas por `sjoin` en GeoPandas son las siguientes.

- **`intersects`**: Cuando las geometrías se cruzan o tocan.

- **`contains`**: Cuando una geometría contiene a otra.

- **`within`**: Cuando una geometría está completamente dentro de otra

**Ejemplo**: Unir un conjunto de puntos con polígonos para determinar en qué polígono se encuentra cada punto.

`result = gpd.sjoin(polygons_gdf, points_gdf, predicate="intersects")`

<br>


In [ ]:
# realizar la unión espacial
celdas_paraderos_join = gpd.sjoin(celdas, paraderos[['geometry']], how="left", predicate="intersects")

# contar los paraderos por celda
paraderos_count = (celdas_paraderos_join.groupby('h3', as_index=False).agg({'index_right':'count'}).rename(columns={'index_right':'n_paraderos'}))

# agregar la columna de conteo al GeoDataFrame de celdas
celdas_censo_paraderos = celdas_censo.merge(paraderos_count, on='h3', how='left')
celdas_censo_paraderos['n_paraderos'] = celdas_censo_paraderos['n_paraderos'].fillna(0)

# visualizar el total de paraderos por celda h3
celdas_censo_paraderos.explore(column='n_paraderos', scheme='NaturalBreaks', k=10)

---

<!-- Slide: Síntesis de Diferencias entre overlay y sjoin -->
<style>
  .reveal table.geo { 
    border-collapse: collapse; 
    margin: 0 auto; 
    width: 100%;
    font-size: 0.9em;
  }
  .reveal table.geo th, 
  .reveal table.geo td { 
    border-top: 1px solid #ccc;
    border-bottom: 1px solid #ccc;
    padding: 6px 8px;
    vertical-align: middle;
  }
  .reveal table.geo th { 
    background-color: #f9f9f9;
  }
  .reveal table.geo td.col-prim,
  .reveal table.geo th.col-prim { 
    text-align: left;
    font-weight: bold;
  }
  .reveal table.geo td.col-ovl,
  .reveal table.geo th.col-ovl,
  .reveal table.geo td.col-sjn,
  .reveal table.geo th.col-sjn {
    text-align: left;
  }
</style>

<h3>8. Síntesis de Diferencias entre <code>overlay</code> y <code>sjoin</code></h3>

<table class="geo">
  <thead>
    <tr>
      <th class="col-prim">Característica</th>
      <th class="col-ovl"><code>overlay</code></th>
      <th class="col-sjn"><code>sjoin</code></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td class="col-prim">Enfoque</td>
      <td class="col-ovl">Crea <strong>nuevas geometrías</strong> basadas en operaciones geométricas (intersección, unión, diferencia, etc.).</td>
      <td class="col-sjn">Combina <strong>atributos</strong> según relaciones espaciales (intersección, contención, etc.) entre puntos y polígonos.</td>
    </tr>
    <tr>
      <td class="col-prim">Relaciones Soportadas</td>
      <td class="col-ovl"><code>intersection</code>, <code>union</code>, <code>difference</code>, <code>symmetric_difference</code>.</td>
      <td class="col-sjn"><code>intersects</code>, <code>contains</code>, <code>within</code>.</td>
    </tr>
    <tr>
      <td class="col-prim">Resultado Típico</td>
      <td class="col-ovl">Un nuevo GeoDataFrame con <strong>geometrías resultantes</strong> de la operación.</td>
      <td class="col-sjn">Un GeoDataFrame con <strong>atributos combinados</strong> sin alterar las geometrías originales.</td>
    </tr>
    <tr>
      <td class="col-prim">Ejemplo en el Laboratorio</td>
      <td class="col-ovl">Distribuir variables como <code>personas</code>, <code>viviendas</code> y <code>hogares</code> proporcionalmente según el área de superposición entre polígonos censales y celdas.</td>
      <td class="col-sjn">Contar cuántos paraderos de transporte público hay en cada celda.</td>
    </tr>
    <tr>
      <td class="col-prim">Cuándo Usar</td>
      <td class="col-ovl">Cuando se necesita <strong>modificar</strong> o <strong>crear nuevas geometrías</strong> basadas en la superposición de dos conjuntos de datos. Para <strong>redistribuir atributos</strong> en función de la <strong>proporción de área</strong> de intersección.</td>
      <td class="col-sjn">Cuando se desea <strong>asociar atributos</strong> de un conjunto de datos a otro basándose en su <strong>relación espacial</strong>. Ideal para <strong>conteos</strong> o asignaciones simples sin necesidad de modificar las geometrías existentes.</td>
    </tr>
  </tbody>
</table>

---

## 9. Análisis de Correlaciones con `seaborn`

Después de integrar tanto los **datos censales** como los **paraderos de transporte público** en las celdas, es útil explorar las **correlaciones** entre las variables para identificar posibles relaciones significativas para nuestros futuros laboratorios.

### Creación de la Matriz de Correlaciones

Utilizaremos la biblioteca `seaborn` para generar una **matriz de correlaciones** que nos permitirá visualizar cómo se relacionan las diferentes variables entre sí.

<br>


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# seleccionar columnas de interés para el análisis de correlación
vars_corr = ['personas', 'hogares', 'viviendas', 'n_paraderos', 'm2_habitacional']

# crear la matriz de correlación
corr_matrix = celdas_censo_paraderos[vars_corr].corr()

# configurar el tamaño de la figura
plt.figure(figsize=(10, 8))

# graficar la matriz de correlación con seaborn
sns.heatmap(corr_matrix, annot=True, cmap='YlGnBu', fmt=".2f")

# añadir título
plt.title("Matriz de Correlaciones")

# mostrar el gráfico
plt.show()

## 10. Guardar los Datos Procesados

Una vez que hemos **integrado la información de los datos censales y los paraderos de transporte público a las celdas**, debemos guardar este GeoDataFrame para su uso en futuros análisis y laboratorios.

<br>

In [ ]:
# exportar el GeoDataFrame 'celdas_censo_paraderos' a un archivo GeoJSON
celdas_censo_paraderos.to_file("celdas.geojson")

**Fin del Laboratorio 2**

---

En el próximo laboratorio, nos enfocaremos en el cálculo de accesibilidades gravitacionales.